In [1]:
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\dell\Downloads\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.drop(columns = ['RowNumber','CustomerId','Surname'],inplace = True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Exited']),df['Exited'],random_state=42,test_size=0.2)

In [9]:
print('count of rows in X_train',X_train.shape)
print('count of rows in X_test',X_test.shape)

count of rows in X_train (8000, 10)
count of rows in X_test (2000, 10)


In [10]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [12]:
num_pipeline = Pipeline([
    ('num_imputation',SimpleImputer(strategy='mean')),
    ('feature_scalling',MinMaxScaler())
])

In [13]:
num_pipeline

Pipeline(steps=[('num_imputation', SimpleImputer()),
                ('feature_scalling', MinMaxScaler())])

In [14]:
import numpy as np

In [16]:
num_cols = df.drop(columns=['Exited']).select_dtypes (include = np.number).columns.tolist()

In [17]:
num_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [18]:
cat_cols = df.drop(columns = ['Exited']).select_dtypes(include = 'object').columns.tolist()

In [19]:
cat_cols

['Geography', 'Gender']

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [22]:
cat_pipeline = Pipeline([
    ('cat_imputation',SimpleImputer(fill_value = 'missing',strategy='constant')),
    ('cat_encoding',OneHotEncoder(sparse_output=False))
])

In [23]:
preprocessor = ColumnTransformer([
    ('categorial',cat_pipeline,cat_cols),
    ('numerical',num_pipeline,num_cols)
])

In [24]:
Pipe = Pipeline([
    ('preprocessor' , preprocessor),
    ('extimator',RandomForestClassifier())
])

In [25]:
Pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorial',
                                                  Pipeline(steps=[('cat_imputation',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('cat_encoding',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['Geography', 'Gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('num_imputation',
                                                                   SimpleImputer()),
                                                                  ('feature_scalling',
                                                                   MinMaxScaler())]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('extimator', RandomForestClassifier())])

In [27]:
Pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorial',
                                                  Pipeline(steps=[('cat_imputation',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('cat_encoding',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['Geography', 'Gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('num_imputation',
                                                                   SimpleImputer()),
                                                                  ('feature_scalling',
                                                                   MinMaxScaler())]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('extimator', RandomForestClassifier())])

In [28]:
Pipe.score(X_test,y_test)*100

86.7

In [30]:
y_pred = Pipe.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
cm = confusion_matrix(y_pred,y_test)

In [34]:
cm

array([[1550,  209],
       [  57,  184]])

In [35]:
(1550+184)/2000*100

86.7